# State preparation

## Definition
State preparation allows us to load states into the state vector. It performs the following unitary operation :

$$\displaystyle 
\large \lvert 0^{\bigotimes n} \rangle 
\xrightarrow{\text{StatePreparation}} \ \ \ \ 
\textstyle\sum\limits_{i=0}^{2^n} a_{i}\displaystyle | i \rangle
$$

where $a$ is a vector filled with amplitudes, of length $2^n$.

## State preparation in pyAQASM
StatePreparation is located in the ``qat.lang.AQASM.oracles`` module.

It takes as parameter a vector of size $2^n$ where $n$ is the number of qbits on which to apply the gate.

Here is a little example using StatePreparation.

In [ ]:
from qat.lang.AQASM import *
from qat.linalg.oracles import StatePreparation

from qat.qpus import LinAlg
from math import sqrt

import numpy

# Creating the vector to be used in StatePreparation
vector = numpy.array([1/sqrt(2), 1/2, 0, 1/2])

# Lets check that the vector represents a valid quantum state
assert numpy.linalg.norm(vector) - 1. < 1e-10

prog = Program()
qbits = prog.qalloc(3)

# Applying StatePreparation
prog.apply(StatePreparation(vector), qbits[0], qbits[2])

# Print the results
circuit = prog.to_circ()
qpu = LinAlg()
result = qpu.submit(circuit.to_job())
for sample in result:
    print("State %s amplitude %s\n" % (sample.state, sample.amplitude))

## State preparation and QFT

Here is another fancier example:

In [ ]:
# We will need a QFT for this example
from qat.lang.AQASM.qftarith import QFT
from matplotlib import pyplot as plt


N = 10
step = 1. / (1 << N)
# lets generate a nice signal
vector = numpy.array([numpy.sin(i * step * 2.* numpy.pi) for i in range(1 << N)])
# And normalize it
vector /= numpy.linalg.norm(vector)

# Lets prepare the state, and apply a QFT 
prog = Program()
qbits = prog.qalloc(10)
prog.apply(StatePreparation(vector), qbits)
prog.apply(QFT(10), qbits)
circuit = prog.to_circ()

# Now we juste need to simulate the circuit and gather the final amplitudes
qpu = LinAlg()
result = qpu.submit(circuit.to_job())
final_state = numpy.zeros(shape=(1 << N))
for sample in result:
    final_state[sample.state.int] = numpy.absolute(sample.amplitude)

# And display everything nicely
plt.plot(vector)
plt.title("Initial signal")
plt.show()
plt.plot(final_state)
plt.title("Fourier transform of the signal")
plt.show()